# Elastic Stackの構築手順：収容（AWS）

----

Elasticsearchのインストール先となる Amazon&nbsp;EC2インスタンスを確保します。  


## 事前条件の確認

**事前に [01_01_Outline.ipynb#設定ファイルの出力](01_01_Outline.ipynb#設定ファイルの出力) を実行しておく必要があります。**


In [ ]:
import os
assert os.path.exists('group_vars/all')

**上記がエラーになる場合は、[01_01_Outline.ipynb#設定ファイルの出力](01_01_Outline.ipynb#設定ファイルの出力)を実行してください。**

> 以下はOperationHubのお試し環境から実施する場合は不要です。

**事前にTerminalから、 `aws configure` を実施してください。**

`aws configure` の実施例:

```
$ aws configure
AWS Access Key ID [None]: (自身のアカウントのアクセスキー)
AWS Secret Access Key [None]: (自身のアカウントのシークレットアクセスキー)
Default region name [None]: ap-northeast-1 (使用したいリージョン)
Default output format [None]: json
```

正しくアクセス情報が指定されているかどうかを、`describe-account-attributes`により確認します。**以下のコマンド実行がエラーとなる場合、`aws configure`が正しくなされていない可能性があります。**

In [ ]:
!aws ec2 describe-account-attributes

## 設定
インスタンスの生成に必要な情報を設定します。

マシンイメージを設定します。

In [ ]:
#CentOS 7 (x86_64) - with Updates HVM
# ap-northeast-1 (アジアパシフィック東京リージョン) を使用する場合のAMI ID
image_id = 'ami-eec1c380'

# us-west-2 (オレゴン) を使用する場合
#image_id = 'ami-f4533694'

インスタンスタイプとインスタンス数を設定します。  
[Amazon EC2 インスタンス](https://aws.amazon.com/jp/ec2/instance-types/)のページから適切なタイプを選択してください。

例えば64GBのメモリを利用するのであれば **m4.4xlarge** が選択候補となります。  
利用したいサイズに応じてタイプを選んで下さい。

In [ ]:
instance_type = 'm4.4xlarge'
instance_count = 1

データ蓄積ディスクとなる[EBS](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/AmazonEBS.html)の[ブロックデバイスマッピング](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/block-device-mapping-concepts.html)設定を行います。  
`VolumeSize`、`VolumeType`を設定します。  
`DeleteOnTermination`にtrueを設定すると、インスタンス破棄時にEBSが削除されます。  


In [ ]:
block_device_mappings = {
    "DeviceName": "/dev/sda1",
    "Ebs": {
      "VolumeSize": 1024,
      "VolumeType": "gp2",
      "DeleteOnTermination": "true",
    }
}

### Keypairの準備

使用するキーペア名を設定します。

In [ ]:
hostname = !hostname
keypair_name = 'es-jupyter-{}'.format(hostname[0])
keypair_name

In [ ]:
import os
ssh_keypath = os.path.expanduser('~/.ssh')
if not os.path.exists(os.path.join(ssh_keypath, 'ansible_id_rsa')):
    !ssh-keygen -t rsa -b 2048 -C ansible -f {ssh_keypath}/ansible_id_rsa -P ''
!cat {ssh_keypath}/ansible_id_rsa.pub

In [ ]:
import json
result = !aws ec2 describe-key-pairs
keypairs = json.loads('\n'.join(result))['KeyPairs']
keypairs

既存キーペアを削除しておく。

In [ ]:
if keypair_name in [k['KeyName'] for k in keypairs]:
    !aws ec2 delete-key-pair --key-name {keypair_name}

In [ ]:
!aws ec2 import-key-pair --key-name {keypair_name} --public-key-material file://{ssh_keypath}/ansible_id_rsa.pub

### VPCの準備

Elasticsearch構築Notebookは、VPCに仮想マシンを確保します。

> 以下はOperationHubのお試し環境から実施する場合を想定し、CloudFormationの出力ファイルからVPC, Subnetを決定しています。

In [ ]:
import json
with open(os.path.expanduser('~/cloudformation-stack.json'), 'r') as f:
    cf_stack = json.load(f)

vpc_id = [c['OutputValue'] for c in cf_stack if c['OutputKey'] == 'OpHubVPC'][0]
subnet_id = [c['OutputValue'] for c in cf_stack if c['OutputKey'] == 'OpHubSubnet'][0]
(vpc_id, subnet_id)

### Security Groupの準備

Security GroupもVPC同様に準備します。

> 以下はOperationHubのお試し環境から実施する場合を想定しています。OperationHubとは異なるSecurity Groupを作成します。

すでに作成してあるSecurity Groupを使用する場合は、以下にIDを設定してください。

In [ ]:
#security_group_id = 'sg-xxxxxxxx'

VPC同様、IDが設定されていない場合は、Security Groupを生成します。自身のIPアドレス(範囲)を正しく設定してください。

In [ ]:
if 'security_group_id' not in locals():
    # このJupyter Notebook Serverが動作している環境からのアクセスのみを許可
    my_sg = [c['OutputValue'] for c in cf_stack if c['OutputKey'] == 'OpHubSecurityGroup'][0]

    result = !aws ec2 describe-security-groups
    sgs = json.loads(''.join(result))['SecurityGroups']
    tagged_sgs = [dict([(t['Key'], t['Value']) for t in sg['Tags']] + \
                       [('GroupId', sg['GroupId']), ('VpcId', sg['VpcId'])])
                   for sg in sgs if 'Tags' in sg and 'VpcId' in sg]
    target_sgs = [sg for sg in tagged_sgs if 'Name' in sg and sg['Name'] == 'es-jupyter' and sg['VpcId'] == vpc_id]
    if len(target_sgs) == 0:
        result = !aws ec2 create-security-group --group-name es-jupyter --description es-jupyter --vpc-id {vpc_id}
        new_sg_id = json.loads(''.join(result))['GroupId']
        print('Security Group created: {}'.format(new_sg_id))
        !aws ec2 create-tags --resources {new_sg_id} --tags Key=Name,Value=es-jupyter
        # SSH
        !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --source-group {my_sg} --protocol tcp --port 22
        # ICMP
        !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --source-group {my_sg} --protocol icmp --port -1
        # Elasticsearch
        !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --source-group {my_sg} --protocol tcp --port 9200
        !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --source-group {new_sg_id} --protocol all
        security_group_id = new_sg_id
    else:
        existing_sg_id = target_sgs[0]['GroupId']
        print('Security Group found: {}'.format(existing_sg_id))
        security_group_id = existing_sg_id
security_group_id

## EC2インスタンスの起動・設定

サーバ構成に応じたEC2インスタンスを起動し、各種設定を実施します。


### インスタンスの起動

ここまでで設定した定義を用いてインスタンスを生成します。

以下では、AWSに対してインスタンス起動要求を発行しています。

> Elasticsearchの試用が終わったら、AWSコンソールからインスタンスを削除するようにしてください。

> **初めてこのNotebookを使ってインスタンスを起動する際**に、CentOSのAMIイメージの利用条件確認を求めるエラーメッセージが表示される場合があります。エラーメッセージ中にAMI MarketplaceのURLが表示されますので、利用条件を確認の上Acceptしてから再度実行してください。

In [ ]:
%run scripts/get-json-repr.py

result = !aws ec2 run-instances --image-id {image_id} \
                                --count {instance_count} \
                                --instance-type {instance_type} \
                                --key-name {keypair_name} \
                                --block-device-mappings '{get_block_device_mappings_repr(block_device_mappings)}' \
                                --security-group-ids {security_group_id} \
                                --subnet-id {subnet_id}
try:
    instances = json.loads('\n'.join(result))['Instances']
except ValueError:
    raise ValueError('\n'.join(result))
instances


生成したインスタンスIDを確認します。

In [ ]:
id_list = map(lambda i: i['InstanceId'], instances)
id_list_text = ' '.join(id_list)
id_list


インスタンスの状態が running になるまで待ちます。

In [ ]:
from functools import reduce
import time

retries = 10
while retries > 0:
    # インスタンス情報を取得
    desc_instances_stdout = !aws ec2 describe-instances --instance-ids $id_list_text
    try:
        json.loads('\n'.join(desc_instances_stdout))
    except:
        print('\n'.join(desc_instances_stdout))
    
    # JSON化し、インスタンスのリストとして取り出す
    desc_instances = json.loads('\n'.join(desc_instances_stdout))
    target_instances = reduce(lambda x, y: x + y, map(lambda r: r['Instances'], desc_instances['Reservations']))
    # IDと状態を出力
    status_text = '\n'.join(map(lambda i : '{0} : {1}'.format(i['InstanceId'], i['State']['Name']), target_instances))
    print(status_text)
    # runningになっていないものがまだあれば、スリープして最初に戻る。全てrunningなら終了。
    not_running_num = len(list(filter(lambda i : i['State']['Name'] != 'running', target_instances)))
    if not_running_num == 0:
        break
    print('Not running : {0}'.format(not_running_num))
    time.sleep(5)
    print('------')
    retries -= 1

for i in target_instances:
    assert i['State']['Name'] == 'running'
host_list = [(i['InstanceId'], i['PrivateIpAddress'], i['PrivateIpAddress']) for i in target_instances]
host_list

### タグの設定

タグのNameキーに名称を設定します。

In [ ]:
!aws ec2 create-tags --resources {id_list_text} --tags Key=Name,Value=es-jupyter

### Elastic IPの設定

インスタンスにElastic IPを設定します。

In [ ]:
host_list = []
for i in target_instances:
    result = !aws ec2 allocate-address
    new_eip = json.loads(''.join(result))
    !aws ec2 associate-address --allocation-id {new_eip['AllocationId']} --instance-id {i['InstanceId']}
    # OperationHubで実行することを想定して、全てPrivate Addressで通信する ... EIPを持たせるのは、外部との通信のため
    host_list.append((i['InstanceId'], i['PrivateIpAddress'], i['PrivateIpAddress']))
host_list

pingが通ることを確認します。

Instanceがrunningになった後、**<font color="red">pingが疎通するようになるまでしばらく(数分程度)</font>**かかる場合もあります。pingの実行が成功する(packet lossが0%)まで以下のセルの実行を試すようにしてください。

In [ ]:
for _, host, _ in host_list:
    !ping -c 4 {host}

全てのインスタンスに対しpingが通ればEC2インスタンス生成は完了です。

### Host Keyの確認


In [ ]:
default_username = 'centos'
default_username

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

In [ ]:
%%writefile {work_dir}/init-password-aws
#!/usr/bin/expect

set host [lindex $argv 0]
set key [lindex $argv 1]
set username [lindex $argv 2]

spawn env LANG=C ssh -i $key "${username}\@${host}"

expect {
    -glob "(yes/no/*)?" {
        send "yes\n"
        exp_continue
    }
}

exit 0



In [ ]:
import time

# VMのsshserverが落ち着くまで待つ...
time.sleep(30)

In [ ]:
for _, host, _ in host_list:
    !ssh-keygen -R {host} || exit 0
    !expect {work_dir}/init-password-aws {host} ~/.ssh/ansible_id_rsa {default_username}

## セキュリティグループへのホスト情報の登録

各インスタンスのPrivate IPアドレスを、セキュリティグループの許可リストに追加します。

インスタンスごとに、9200番ポート（外部通信用）と9300番ポート（クラスタ内部通信用）の両方を許可します。

In [ ]:
!aws ec2 authorize-security-group-ingress --group-id {security_group_id} --protocol tcp --port 9200 --source-group {security_group_id}
!aws ec2 authorize-security-group-ingress --group-id {security_group_id} --protocol tcp --port 9300 --source-group {security_group_id}

## Inventory作成用のインスタンスリストを作成
ホストのパブリックDNS一覧をリスト化します  

次のブックの[Inventory作成](01_03_Set_Inventory.ipynb)で使用します。  
出現した文字列をクリップボードにコピーしてください。

In [ ]:
print('host_list = {}'.format(repr(dict([('host{}'.format(i + 1), h) for i, h in enumerate(host_list)]))))